In [1]:
from array import array
import reprlib
import math
import numbers

In [2]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

# BEGIN VECTOR_V3_GETATTR
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # <1>
        if len(name) == 1:  # <2>
            pos = cls.shortcut_names.find(name)  # <3>
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))
# END VECTOR_V3_GETATTR

# BEGIN VECTOR_V3_SETATTR
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # <1>
            if name in cls.shortcut_names:  # <2>
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3>
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # <4>
            if error:  # <5>
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # <6>

# END VECTOR_V3_SETATTR

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

#### A multi-dimensional ``Vector`` class, take 3
#### A ``Vector`` is built from an iterable of numbers::

In [3]:
>>> Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [4]:
>>> Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
>>> Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

#### Tests with 2-dimensions (same results as ``vector2d_v1.py``)::

In [6]:
>>> v1 = Vector([3, 4])
>>> x, y = v1
>>> x, y

(3.0, 4.0)

In [7]:
>>> v1

Vector([3.0, 4.0])

In [8]:
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone

True

In [9]:
>>> print(v1)

(3.0, 4.0)


In [10]:
>>> octets = bytes(v1)
>>> octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [11]:
>>> abs(v1)

5.0

In [12]:
>>> bool(v1), bool(Vector([0, 0]))

(True, False)

#### Test of ``.frombytes()`` class method:

In [13]:
>>> v1_clone = Vector.frombytes(bytes(v1))
>>> v1_clone

Vector([3.0, 4.0])

In [14]:
>>> v1 == v1_clone

True

#### Tests with 3-dimensions::

In [15]:
>>> v1 = Vector([3, 4, 5])
>>> x, y, z = v1
>>> x, y, z

(3.0, 4.0, 5.0)

In [16]:
>>> v1

Vector([3.0, 4.0, 5.0])

In [17]:
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone

True

In [18]:
>>> print(v1)

(3.0, 4.0, 5.0)


In [19]:
>>> abs(v1)  # doctest:+ELLIPSIS

7.0710678118654755

In [20]:
>>> bool(v1), bool(Vector([0, 0, 0]))

(True, False)

#### Tests with many dimensions::

In [21]:
>>> v7 = Vector(range(7))
>>> v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [22]:
>>> abs(v7)  # doctest:+ELLIPSIS

9.539392014169456

#### Test of ``.__bytes__`` and ``.frombytes()`` methods::

In [23]:
>>> v1 = Vector([3, 4, 5])
>>> v1_clone = Vector.frombytes(bytes(v1))
>>> v1_clone

Vector([3.0, 4.0, 5.0])

In [24]:
>>> v1 == v1_clone

True

#### Tests of sequence behavior::

In [25]:
>>> v1 = Vector([3, 4, 5])
>>> len(v1)

3

In [26]:
>>> v1[0], v1[len(v1)-1], v1[-1]

(3.0, 5.0, 5.0)

#### Test of slicing::

In [27]:
>>> v7 = Vector(range(7))
>>> v7[-1]

6.0

In [28]:
>>> v7[1:4]

Vector([1.0, 2.0, 3.0])

In [29]:
>>> v7[-1:]

Vector([6.0])

In [30]:
>>> v7[1,2]

TypeError: Vector indices must be integers

#### Tests of dynamic attribute access::

In [31]:
>>> v7 = Vector(range(10))
>>> v7.x

0.0

In [32]:
>>> v7.y, v7.z, v7.t

(1.0, 2.0, 3.0)

#### Dynamic attribute lookup failures::

In [33]:
>>> v7.k

AttributeError: 'Vector' object has no attribute 'k'

In [34]:
>>> v3 = Vector(range(3))
>>> v3.t

AttributeError: 'Vector' object has no attribute 't'

In [35]:
>>> v3.spam

AttributeError: 'Vector' object has no attribute 'spam'

#### Tests of preventing attributes from 'a' to 'z'::

In [36]:
>>> v1.x = 7

AttributeError: readonly attribute 'x'

In [37]:
>>> v1.w = 7

AttributeError: can't set attributes 'a' to 'z' in 'Vector'

#### Other attributes can be set::

In [38]:
>>> v1.X = 'albatross'
>>> v1.X

'albatross'

In [39]:
>>> v1.ni = 'Ni!'
>>> v1.ni

'Ni!'